Detect motion with PIL based on the recipe for OpenCV in the reference. This is because installing OpenCV on Raspberry Pi is a bear.

In [2]:
from picamera import PiCamera
from io import BytesIO
from PIL import Image, ImageFilter, ImageMorph, ImageEnhance
import time
import os